In [4]:
import torch
import torch.nn as nn
import numpy as np

class NeuralAcousticField(nn.Module):
    def __init__(self, hidden_dim=256):
        super().__init__()
        # Теперь 6 входных признаков: x,y,z источника + x,y,z слушателя
        self.net = nn.Sequential(
            nn.Linear(6, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 3)  # amplitude, delay, reverb_decay
        )

    def forward(self, coords):
        return self.net(coords)

In [5]:
def generate_training_data(num_samples=1000):
    """Генерация данных: source_pos + listener_pos = 6 координат"""
    sources = np.random.rand(num_samples, 3) * [10, 10, 3]  # x,y,z источника
    listeners = np.random.rand(num_samples, 3) * [10, 10, 3] # x,y,z слушателя

    # Физическая модель (упрощенная)
    distances = np.linalg.norm(sources - listeners, axis=1)
    amplitudes = 1.0 / (distances + 1e-5)
    delays = distances / 343.0  # Скорость звука
    reverbs = 0.5 * np.exp(-distances/5.0)

    # Входные координаты (6D: source_x,y,z + listener_x,y,z)
    coords = np.concatenate([sources, listeners], axis=1)

    return torch.FloatTensor(coords), torch.FloatTensor(np.column_stack((
        amplitudes, delays, reverbs
    )))

In [6]:
# Инициализация
model = NeuralAcousticField()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Данные (5000 samples, 6 координат)
X, y = generate_training_data(5000)

# Обучение
for epoch in range(1000):
    optimizer.zero_grad()
    preds = model(X)
    loss = criterion(preds, y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 0.3620
Epoch 100, Loss: 0.0093
Epoch 200, Loss: 0.0049
Epoch 300, Loss: 0.0032
Epoch 400, Loss: 0.0019
Epoch 500, Loss: 0.0012
Epoch 600, Loss: 0.0007
Epoch 700, Loss: 0.0004
Epoch 800, Loss: 0.0003
Epoch 900, Loss: 0.0002


In [7]:
def predict_acoustics(model, source_pos, listener_pos):
    coords = torch.FloatTensor(np.concatenate([
        source_pos,  # [x,y,z] источника
        listener_pos  # [x,y,z] слушателя
    ])).unsqueeze(0)  # Добавляем batch-размер

    with torch.no_grad():
        amp, delay, reverb = model(coords)[0]

    print(f"Amplitude: {amp.item():.2f}")
    print(f"Delay: {delay.item():.4f}s")
    print(f"Reverb decay: {reverb.item():.2f}")

# Пример вызова
predict_acoustics(model, [2.0, 3.0, 1.5], [5.0, 4.0, 1.5])

Amplitude: 0.32
Delay: 0.0068s
Reverb decay: 0.25
